In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from dotenv import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, PDFSearchTool, WebsiteSearchTool, ScrapeWebsiteTool, DirectoryReadTool, FileReadTool, CodeInterpreterTool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
import DataAcquisition, DataProcessing, BuildTradingStrategy, BackTest, MarketAnalysis# AgentRoles.py
from crewai import Agent
from langchain.tools import Tool
# from langchain.utilities import PythonREPL

# class TradingAgents:
#     def __init__(self):
#         self.python_repl = PythonREPL()
        
#     def market_analyst(self):
#         return Agent(
#             role="Senior Market Analyst",
#             goal="Identify high-probability trading opportunities through technical and fundamental analysis",
#             backstory="Expert in quantitative analysis and pattern recognition with 15+ years market experience",
#             tools=[
#                 Tool(
#                     name="technical_analysis",
#                     func=self._run_technical_analysis,
#                     description="Performs technical analysis using TA-Lib indicators"
#                 ),
#                 Tool(
#                     name="fundamental_evaluation",
#                     func=self._run_fundamental_analysis,
#                     description="Evaluates fundamental factors using financial statements and economic data"
#                 )
#             ],
#             verbose=True
#         )

#     def risk_manager(self):
#         return Agent(
#             role="Chief Risk Officer",
#             goal="Maintain portfolio risk within acceptable parameters",
#             backstory="Former hedge fund risk director specializing in volatility management",
#             tools=[
#                 Tool(
#                     name="var_calculator",
#                     func=self._calculate_var,
#                     description="Calculates Value-at-Risk using historical simulation method"
#                 )
#             ],
#             verbose=True
#         )

#     def execution_engineer(self):
#         return Agent(
#             role="Lead Execution Engineer",
#             goal="Optimize trade execution and minimize market impact",
#             backstory="Low-latency trading systems architect with expertise in order routing",
#             tools=[
#                 Tool(
#                     name="smart_order_router",
#                     func=self._route_order,
#                     description="Routes orders to optimal liquidity venues using ML-based routing logic"
#                 )
#             ],
#             verbose=True
#         )

#     def _run_technical_analysis(self, query):
#         return self.python_repl.run(f"from MarketAnalysis import technical_indicators; technical_indicators.{query}")

#     def _run_fundamental_analysis(self, query):
#         return self.python_repl.run(f"from MarketAnalysis import fundamental_evaluation; fundamental_evaluation.{query}")

#     def _calculate_var(self, params):
#         return self.python_repl.run(f"from RiskManagement import calculate_var; calculate_var({params})")

#     def _route_order(self, order_details):
#         return self.python_repl.run(f"from ExecutionEngine import smart_order_router; smart_order_router({order_details})")
# from langchain_community.utilities.sql_database import SQLDatabase

In [8]:
import os
import yaml
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# from langchain.chat_models import init_chat_model

# codellm = init_chat_model("ollama/codestral:latest", model_provider="ollama", base_url="http://localhost:11434")
# minillm = init_chat_model("ollama/nemotron-mini",model_provider="ollama",base_url="http://localhost:11434")
# llm = init_chat_model("ollama/phi4",model_provider="ollama",base_url="http://localhost:11434")
# prollm = init_chat_model("ollama/deepseek-r1:70b",model_provider="ollama",base_url="http://localhost:11434")

In [10]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']

In [11]:
import os
from dotenv import load_dotenv
from crewai.tools import BaseTool
from pydantic import Field
from langchain_ollama.llms import OllamaLLM
 




# Now initialize your agent using the Groq-backed model (and your wrapped tools)
from crewai import Agent, Crew
from DataAcquisition import DataAcquisitionTasks
from crewai_tools import ScrapeWebsiteTool, WebsiteSearchTool, SerperDevTool, DirectoryReadTool, FileReadTool, CodeInterpreterTool, CodeDocsSearchTool
from langchain_community.tools import TavilySearchResults

# from langchain_openai import ChatOpenAI
# codellm = ChatOpenAI(
#     model="ollama/codestral",
#     base_url="http://localhost:11434",
#     custom_llm_provider="ollama",
#     temperature=0.1)

alpha_vantage = AlphaVantageAPIWrapper()
#search = GoogleSerperAPIWrapper()

class AlphaVantageTool(
    ool):
    name: str = "alpha_vantage"
    description: str = "Useful to query financial data using AlphaVantage API"
    alpha_vantage: AlphaVantageAPIWrapper = Field(default_factory=AlphaVantageAPIWrapper)

    def _run(self, query: str) -> str:
        """Call Alpha Vantage API and return results"""
        try:
            return self.alpha_vantage.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"
# alpha_vantage_documentation = 'https://www.alphavantage.co/documentation/'
# alpha_vantage._get_exchange_rate("USD", "JPY")
#from FinBot.AlphaVantageTool import AlphaVantageTool  # Wrapped tool for AlphaVantage
#from langchain_community.llms import Ollama

# codellm = OllamaLLM(
#     model="codestral:latest",  # just the model name without ollama/
#     # model_provider="ollama",
#     # base_url="http://localhost:11434"
# )
#codellm = LLM(model="ollama/codestral:latest", model_provider="ollama", base_url="http://localhost:11434")
# llm = Ollama(
#         model="ollama/codestral:latest",
#         base_url="http://localhost:11434",
#         #temperature=0.1
#         #custom_llm_provider="ollama"
#     )
# llm.custom_llm_provider = "ollama"


In [12]:
codellm=LLM(model="ollama/codestral", base_url="http://localhost:11434")
llm = LLM(model="ollama/phi4", base_url="http://localhost:11434")
prollm=LLM(model="ollama/deepseek-r1:70b", base_url="http://localhost:11434")
minillm=LLM(model="ollama/nemotron-mini", base_url="http://localhost:11434")

In [13]:
llm.call("Write me hello world in python")

'Certainly! Here is a simple "Hello, World!" program written in Python:\n\n```python\nprint("Hello, World!")\n```\n\nYou can run this code by saving it into a file with a `.py` extension (e.g., `hello_world.py`) and executing it using a Python interpreter from the command line like so: \n\n```bash\npython hello_world.py\n```\n\nOr if you are using Python 3 specifically, you might need to use:\n\n```bash\npython3 hello_world.py\n```\n\nThis will output:\n\n```\nHello, World!\n```'

In [14]:
data_agent = Agent(
    config=agents_config['Data Acquisition Agent'],
    verbose=True,
    llm=llm,
    tools=[
        #ScrapeWebsiteTool(),
        #WebsiteSearchTool(),
        #AlphaVantageTool(),  # Using the wrapped tool
        #TavilySearchResults(max_results=10),
        SerperDevTool(),
        #DirectoryReadTool(),
        #FileReadTool(),
        CodeInterpreterTool(),
        #CodeDocsSearchTool()
    ],
    allow_delegation=True,
    allow_code_execution=True
)

query_builder = Agent(
    role="API Expert",
    goal="Go through API documentation and Build a query to retrieve all the requested datapoints for {instrument_name} from AlphaVantage",
    backstory="You are an expert in the field of financial data retrieval and analysis. You have a deep understanding of the AlphaVantage API and its capabilities.",
    tools=[AlphaVantageTool(), CodeInterpreterTool()],
    llm=minillm,
    verbose=True,
    allow_delegation=False,
    allow_code_execution=True
)

    

# Define common parameters for your data acquisition tasks
asset_class = "stocks"
instrument_name = "AAPL"
market = "US"

# Instantiate your DataAcquisitionTasks manager and create tasks
acquisition_tasks = DataAcquisitionTasks()
task_fetch_ohlc = acquisition_tasks.fetch_multi_frequency_ohlc(data_agent, asset_class, instrument_name, market)
task_build_query = acquisition_tasks.build_query(query_builder, asset_class, instrument_name)
# task_stream_ticks = acquisition_tasks.stream_real_time_ticks(data_agent, asset_class, instrument_name, market)
# task_orderbook = acquisition_tasks.collect_orderbook_option_data(data_agent, asset_class, instrument_name, market)
#task_scrape_news = acquisition_tasks.scrape_news_sentiment(data_agent, asset_class, instrument_name, market)
# task_corporate = acquisition_tasks.ingest_corporate_actions(data_agent, asset_class, instrument_name, market)
#task_economic = acquisition_tasks.fetch_economic_alt_data(data_agent, asset_class, market)
#task_compliance = acquisition_tasks.validate_and_log_compliance(data_agent, asset_class, market)

# Create the Crew and register tasks
crew = Crew(
    agents=[data_agent,query_builder],
    tasks=[
        task_fetch_ohlc,
        task_build_query,
        # task_stream_ticks,
        # task_orderbook,
        #task_scrape_news,
        #task_corporate,
        #task_economic,
        #task_compliance
    ]
)

# Kickoff the tasks via CrewAI orchestration
crew.kickoff()

RuntimeError: Docker is not running. Please start Docker to use code execution with agent: Market Data Retrieval Specialist
